<a href="https://colab.research.google.com/github/GVSU-CIS635/gvsu-cis635-term-project-team_crime-prediction/blob/main/with_layers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates

In [52]:
training_data = pd.read_csv('https://raw.githubusercontent.com/GVSU-CIS635/gvsu-cis635-term-project-team_crime-prediction/main/test2.csv')


In [53]:
training_data.head()

,CATEGORY,CALL GROUPS,final_case_type,CASE DESC,occ_date,x_coordinate,y_coordinate,census_tract
0,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,3/1/12,7641076,684831,4900.0
1,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,3/1/12,7642640,683167,10600.0
2,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,3/1/12,7643599,683216,10600.0
3,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,3/1/12,7644359,693642,3502.0
4,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,3/1/12,7644771,683859,10600.0


In [54]:
# Define the grid dimensions
x_min, x_max = 7603950, 7717500
y_min, y_max = 651190, 733990
num_cells_x = int((x_max - x_min) / 600)
num_cells_y = int((y_max - y_min) / 600)



In [55]:
def show_graph ():
    # Plot 2D heatmap for the current layer
    for z in range(z_max):
        fig, ax = plt.subplots(figsize=(10, 6))

        im = ax.imshow(count_per_cell[:, :, z], cmap='viridis', extent=[x_min, x_max, y_min, y_max], origin='lower', aspect='auto')

        ax.set_title(f'Layer {z + 1}')
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')
        ax.set_title(f"Time Interval: {layer_start_dates[z].strftime('%Y-%m-%d')} to {layer_end_dates[z].strftime('%Y-%m-%d')}")

        # Add colorbar
        fig.colorbar(im, ax=ax, orientation='vertical', pad=0.1)

        plt.show()

def calcualte_PAI(n, N, a, A):
    return (n/N)/(a/A)
def calculate_PEI(PAI,n,N,a,A):
    return PAI/((n/N)/(a/A))

def print_cells():
    for z in range(z_max):
        print(f"Top 100 cells in Layer {z + 1}:")
        for i, (x_idx, y_idx, count) in enumerate(top_cells_per_layer[z], 1):
            print(f"  {i}. Cell ({x_idx + 1}, {y_idx + 1}) - Count: {count}")
        print("\n")
def countcells(points):
  indices_x = np.digitize(points[:, 0], np.linspace(x_min, x_max, num_cells_x))
  indices_y = np.digitize(points[:, 1], np.linspace(y_min, y_max, num_cells_y))

  count_per_cell = np.zeros((num_cells_x, num_cells_y))

  for i in range(len(points)):
      x_idx = indices_x[i] - 1
      y_idx = indices_y[i] - 1

      count_per_cell[x_idx, y_idx] += 1
  return count_per_cell
# Convert 'occ_date' column to ordinal format
training_data['occ_date'] = pd.to_datetime(training_data['occ_date'], format='%m/%d/%y')
training_data['occ_date'] = training_data['occ_date'].apply(mdates.date2num)

# Assign points to the grid
points = training_data[['x_coordinate', 'y_coordinate', 'occ_date']].values
count_per_cell_train = countcells(points)



In [31]:
flat_count = count_per_cell_train.flatten()

# Get the indices of the top 100 cells
top_indices = np.argpartition(flat_count, -100)[-100:]

# Convert flattened indices to 2D indices
top_indices_2d = np.unravel_index(top_indices, count_per_cell_train.shape)

# Create a list of tuples containing (x_index, y_index, count) for the top cells
top_cells_train = [(top_indices_2d[0][i], top_indices_2d[1][i], flat_count[top_indices[i]]) for i in range(len(top_indices))]

# Display the top 100 cells
print("Top 100 cells:")
for i, (x_idx, y_idx, count) in enumerate(top_cells_train, 1):
    print(f"  {i}. Cell ({x_idx + 1}, {y_idx + 1}) - Count: {count}")


Top 100 cells:
  1. Cell (99, 57) - Count: 151.0
  2. Cell (147, 42) - Count: 152.0
  3. Cell (65, 54) - Count: 152.0
  4. Cell (78, 60) - Count: 152.0
  5. Cell (143, 51) - Count: 193.0
  6. Cell (69, 55) - Count: 579.0
  7. Cell (64, 60) - Count: 164.0
  8. Cell (109, 39) - Count: 233.0
  9. Cell (69, 54) - Count: 266.0
  10. Cell (59, 56) - Count: 215.0
  11. Cell (143, 56) - Count: 176.0
  12. Cell (68, 59) - Count: 165.0
  13. Cell (75, 62) - Count: 174.0
  14. Cell (68, 58) - Count: 311.0
  15. Cell (126, 38) - Count: 219.0
  16. Cell (68, 57) - Count: 189.0
  17. Cell (116, 59) - Count: 546.0
  18. Cell (68, 56) - Count: 248.0
  19. Cell (68, 55) - Count: 501.0
  20. Cell (62, 51) - Count: 210.0
  21. Cell (113, 52) - Count: 174.0
  22. Cell (67, 108) - Count: 240.0
  23. Cell (108, 54) - Count: 154.0
  24. Cell (108, 52) - Count: 153.0
  25. Cell (143, 50) - Count: 368.0
  26. Cell (77, 59) - Count: 153.0
  27. Cell (113, 43) - Count: 225.0
  28. Cell (77, 61) - Count: 567.0
  

In [32]:
test_data = pd.read_csv('https://raw.githubusercontent.com/GVSU-CIS635/gvsu-cis635-term-project-team_crime-prediction/main/test3.csv')
test_data.head()

,CATEGORY,CALL GROUPS,final_case_type,CASE DESC,occ_date,x_coordinate,y_coordinate,census_tract
0,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,1/1/13,7625659,714714,4101.0
1,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,1/1/13,7628093,655826,6404.0
2,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,1/1/13,7631900,704307,4002.0
3,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,1/1/13,7643655,682985,10600.0
4,STREET CRIMES,DISORDER,DISTP,DISTURBANCE - PRIORITY ...,1/1/13,7643778,681457,10600.0


In [33]:
start_date = pd.to_datetime('01/01/13', format='%m/%d/%y')
end_date = pd.to_datetime('12/31/13', format='%m/%d/%y')

# Convert 'occ_date' column to datetime format
test_data['occ_date'] = pd.to_datetime(test_data['occ_date'], format='%m/%d/%y')

# Filter the data based on the date range
filtered_data = test_data[(test_data['occ_date'] >= start_date) & (test_data['occ_date'] <= end_date)]

# Convert 'occ_date' column to numerical format if needed
filtered_data['occ_date'] = filtered_data['occ_date'].apply(mdates.date2num)

# Assign points to the grid
points = filtered_data[['x_coordinate', 'y_coordinate', 'occ_date']].values
count_per_cell_test = countcells(points)


In [34]:
flat_count = count_per_cell_test.flatten()

# Get the indices of the top 100 cells
top_indices = np.argpartition(flat_count, -100)[-100:]

# Convert flattened indices to 2D indices
top_indices_2d = np.unravel_index(top_indices, count_per_cell_test.shape)

# Create a list of tuples containing (x_index, y_index, count) for the top cells
top_cells_test = [(top_indices_2d[0][i], top_indices_2d[1][i], flat_count[top_indices[i]]) for i in range(len(top_indices))]

# Display the top 100 cells
print("Top 100 cells:")
for i, (x_idx, y_idx, count) in enumerate(top_cells_test, 1):
    print(f"  {i}. Cell ({x_idx + 1}, {y_idx + 1}) - Count: {count}")

Top 100 cells:
  1. Cell (58, 56) - Count: 170.0
  2. Cell (110, 72) - Count: 173.0
  3. Cell (138, 51) - Count: 173.0
  4. Cell (108, 52) - Count: 174.0
  5. Cell (108, 30) - Count: 224.0
  6. Cell (67, 52) - Count: 249.0
  7. Cell (67, 53) - Count: 700.0
  8. Cell (67, 54) - Count: 351.0
  9. Cell (67, 55) - Count: 572.0
  10. Cell (67, 56) - Count: 530.0
  11. Cell (108, 21) - Count: 247.0
  12. Cell (67, 57) - Count: 544.0
  13. Cell (63, 51) - Count: 543.0
  14. Cell (63, 52) - Count: 215.0
  15. Cell (67, 58) - Count: 414.0
  16. Cell (67, 59) - Count: 239.0
  17. Cell (67, 87) - Count: 247.0
  18. Cell (137, 42) - Count: 414.0
  19. Cell (62, 56) - Count: 212.0
  20. Cell (62, 55) - Count: 327.0
  21. Cell (113, 43) - Count: 244.0
  22. Cell (108, 39) - Count: 446.0
  23. Cell (62, 51) - Count: 217.0
  24. Cell (127, 62) - Count: 206.0
  25. Cell (66, 88) - Count: 256.0
  26. Cell (89, 48) - Count: 204.0
  27. Cell (74, 60) - Count: 238.0
  28. Cell (90, 39) - Count: 214.0
  29.

In [35]:
# Calculate the sum of counts using indices from the last layer
total_top_count_last_layer = np.sum(count_per_cell_test[cell[0], cell[1]] for cell in top_cells_train)

# Calculate the sum of counts for the actual top 100 cells of the next layer
total_top_count_actual = np.sum(top_cells_test)

sum_of_train = np.sum(count_per_cell_train)
sum_of_test = np.sum(count_per_cell_test)
print(f"  Total count using indices from the last layer: {total_top_count_last_layer}")
print(f"  Total count for the actual top 100 cells: {total_top_count_actual}")
print(f"  Sum of Points: {sum_of_train} points")
pai = calcualte_PAI(total_top_count_last_layer,sum_of_test,100,num_cells_x*num_cells_y)
pei = calculate_PEI(pai,total_top_count_actual,sum_of_test,100,num_cells_x*num_cells_y)
print(f"  PEI: {pei}\n")

  Total count using indices from the last layer: 28538.0
  Total count for the actual top 100 cells: 43518.0
  Sum of Points: 146927.0 points
  PEI: 0.6557746219954962



<ipython-input-35-850bf3ad4454>:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  total_top_count_last_layer = np.sum(count_per_cell_test[cell[0], cell[1]] for cell in top_cells_train)


In [48]:
training_data = pd.read_csv('https://raw.githubusercontent.com/GVSU-CIS635/gvsu-cis635-term-project-team_crime-prediction/main/test2.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/GVSU-CIS635/gvsu-cis635-term-project-team_crime-prediction/main/test3.csv')


df_data = pd.concat([training_data, test_data], ignore_index=True)
print(
    f"The combined  df_data dataframe has {df_data.shape[0]} samples and {df_data.shape[1]} attributes;"
)

The combined  df_data dataframe has 320973 samples and 8 attributes;


In [49]:
df_data['occ_date'] = pd.to_datetime(df_data['occ_date'], errors='coerce', format='%m/%d/%y')
days_per_layer = 150

# Drop rows with NaN values in the 'occ_date' column
data = df_data.dropna(subset=['occ_date'])

# Check if there are any non-datetime values in the 'occ_date' column
non_datetime_values = df_data['occ_date'][~df_data['occ_date'].apply(isinstance, args=(datetime.datetime,))]
if not non_datetime_values.empty:
    print(f"Warning: Some rows in 'occ_date' column have non-datetime values: {non_datetime_values}")

# Calculate total_days
total_days = (df_data['occ_date'].max() - df_data['occ_date'].min()).days

z_max = total_days // days_per_layer
print(z_max)

4


In [50]:
df_data['occ_date'] = pd.to_datetime(df_data['occ_date'], format='%m/%d/%y')
df_data['occ_date'] = df_data['occ_date'].apply(mdates.date2num)

# Assign points to the grid
points = df_data[['x_coordinate', 'y_coordinate', 'occ_date']].values

indices_x = np.digitize(points[:, 0], np.linspace(x_min, x_max, num_cells_x))
indices_y = np.digitize(points[:, 1], np.linspace(y_min, y_max, num_cells_y))

count_per_cell = np.zeros((num_cells_x, num_cells_y, z_max))

for i in range(len(points)):
    x_idx = indices_x[i] - 1
    y_idx = indices_y[i] - 1
    z_idx = int((points[i, 2] - df_data['occ_date'].min()) / days_per_layer)

    # Ensure z_idx is within bounds
    z_idx = max(0, min(z_idx, z_max - 1))

    count_per_cell[x_idx, y_idx, z_idx] += 1

sum_per_layer = np.sum(count_per_cell, axis=(0, 1))

layer_start_dates = [mdates.num2date(df_data['occ_date'].min() + days_per_layer * i) for i in range(z_max)]
layer_end_dates = [mdates.num2date(df_data['occ_date'].min() + days_per_layer * (i + 1)) for i in range(z_max)]
top_cells_per_layer = []

for z in range(z_max):
    # Flatten the 2D array for the current layer
    flat_count_layer = count_per_cell[:, :, z].flatten()

    # Get the indices of the top 100 cells
    top_indices = np.argpartition(flat_count_layer, -100)[-100:]

    # Convert flattened indices to 2D indices
    top_indices_2d = np.unravel_index(top_indices, count_per_cell[:, :, z].shape)

    # Create a list of tuples containing (x_index, y_index, count) for the top cells
    top_cells = [(top_indices_2d[0][i], top_indices_2d[1][i], flat_count_layer[top_indices[i]]) for i in range(len(top_indices))]

    # Add the list of top cells for the current layer to the result
    top_cells_per_layer.append(top_cells)


# Print the sum of points in each layer
for z in range(z_max):
    print(f"Layer {z + 1} - Time Interval: {layer_start_dates[z].strftime('%Y-%m-%d')} to {layer_end_dates[z].strftime('%Y-%m-%d')}")

    if z == 0:
        # For the first layer, print the sum of points only
        total_top_count_actual = sum(cell[2] for cell in top_cells_per_layer[z])
        print(f"  Total count for the actual top 100 cells: {total_top_count_actual}")
        print(f"  Sum of Points: {int(sum_per_layer[z])} points\n")
    else:
        # Calculate the sum of counts using indices from the last layer
        total_top_count_last_layer = sum(count_per_cell[cell[0], cell[1], z] for cell in top_cells_per_layer[z - 1])

        # Calculate the sum of counts for the actual top 100 cells of the next layer
        total_top_count_actual = sum(cell[2] for cell in top_cells_per_layer[z])

        print(f"  Total count using indices from the last layer: {total_top_count_last_layer}")
        print(f"  Total count for the actual top 100 cells: {total_top_count_actual}")
        print(f"  Sum of Points: {int(sum_per_layer[z])} points")
        pai = calcualte_PAI(total_top_count_last_layer,int(sum_per_layer[z]),100,num_cells_x*num_cells_y)
        pei = calculate_PEI(pai,total_top_count_actual,int(sum_per_layer[z]),100,num_cells_x*num_cells_y)
        print(f"  PEI: {pei}\n")

Layer 1 - Time Interval: 2012-03-01 to 2012-07-29
  Total count for the actual top 100 cells: 12221.0
  Sum of Points: 72140 points

Layer 2 - Time Interval: 2012-07-29 to 2012-12-26
  Total count using indices from the last layer: 11872.0
  Total count for the actual top 100 cells: 12261.0
  Sum of Points: 72361 points
  PEI: 0.968273387162548

Layer 3 - Time Interval: 2012-12-26 to 2013-05-25
  Total count using indices from the last layer: 10752.0
  Total count for the actual top 100 cells: 11274.0
  Sum of Points: 66069 points
  PEI: 0.9536987759446512

Layer 4 - Time Interval: 2013-05-25 to 2013-10-22
  Total count using indices from the last layer: 18354.0
  Total count for the actual top 100 cells: 19051.0
  Sum of Points: 110403 points
  PEI: 0.9634139940160622

